1) Написать программу, которая собирает входящие письма из своего или тестового почтового ящика
и сложить данные о письмах в базу данных
(от кого, дата отправки, тема письма, текст письма полный)
Логин тестового ящика: study.ai_172@mail.ru
Пароль тестового ящика: NewPassword172

In [1]:
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains as ac
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time
import pendulum
from pprint import pprint
import pandas as pd

In [2]:
login = 'study.ai_172@mail.ru'
password = 'NextPassword172'
count = 0

In [3]:
def get_date(text):
    if 'Вчера' in text:
        return pendulum.yesterday('Europe/Moscow').format('DD.MM.YYYY')
    if 'Сегодня' in text:
        return pendulum.today('Europe/Moscow').format('DD.MM.YYYY')
    date_split= text.split(',')[0].split()
    month_list = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня','июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']
    month = [i+1 for i in range (0,len(month_list)) if date_split[1]==month_list[i]] # не пошло?
    if len(date_split)==2:
        return f'{date_split[0]}.{month[0]}.{pendulum.now().year}'
    if len(date_split)==3:
        return f'{date_split[0]}.{month[0]}.{date_split[2]}'

In [4]:
def get_data_from_letter(driver):
    data_letter = {}
    data_letter['data'] = get_date(WebDriverWait(driver, 20).until(ec.presence_of_element_located((By.CLASS_NAME, 'letter__date'))).text)
    data_letter['from'] = WebDriverWait(driver, 20).until(ec.presence_of_element_located((By.CLASS_NAME, 'letter-contact'))).get_attribute('title')
    data_letter['subject'] = WebDriverWait(driver, 20).until(ec.presence_of_element_located((By.TAG_NAME, 'h2'))).text
    data_letter['letter'] = WebDriverWait(driver, 20).until(ec.presence_of_element_located((By.CLASS_NAME, 'letter-body'))).text
    return data_letter

In [5]:
list_letters=[]
driver=webdriver.Chrome('/usr/local/bin/chromedriver')
driver.get('https://mail.ru')

In [6]:
elem = driver.find_element_by_id('mailbox:login')

In [7]:
elem.send_keys(login)
elem.send_keys(Keys.RETURN)

In [8]:
elem = driver.find_element_by_id('mailbox:password')
elem.send_keys(password)
elem.send_keys(Keys.RETURN)

In [9]:
letter_ = WebDriverWait(driver,20).until(ec.presence_of_element_located((By.CLASS_NAME,'js-tooltip-direction_letter-bottom')))

In [10]:
try:
    driver.get(letter_.get_attribute('href'))
    list_letters.append(get_data_from_letter(driver))
    elem = WebDriverWait(driver,20).until(ec.presence_of_element_located((By.CLASS_NAME,'portal-menu-element_next')))
except  Exception as e:
    print(e,'Писем нет')

In [11]:
while count<117: #кол-во определил по df 
    elem.click()
    time.sleep(3)
    list_letters.append(get_data_from_letter(driver))
    elem =  WebDriverWait(driver,20).until(ec.presence_of_element_located((By.CLASS_NAME,'portal-menu-element_next')))
    count+=1
pprint(list_letters)

[{'data': '30.07.2020',
  'from': 'security@id.mail.ru',
  'letter': '        Вход с нового устройства в аккаунт\n'
            ' study.ai_172@mail.ru \n'
            '      В аккаунт study.ai_172@mail.ru вошли с нового устройства.\n'
            '  Если вы этого не делали, измените пароль, чтобы обезопасить '
            'аккаунт.\n'
            '  Время 30 июля, 17:44\n'
            '  Устройство Ubuntu, Mozilla Firefox 79, Москва, Россия\n'
            '  Посмотреть список устройств\n'
            '  Узнайте, как защитить аккаунт, на нашем сайте по безопасности.\n'
            '    Вы получили это письмо, потому что являетесь пользователем '
            'Mail\u200c.ru на основании Пользовательского соглашения.\n'
            '  Copyright 2020 Mail\u200c.ru Group, Москва — Все права '
            'защищены.\n'
            '  Служба поддержки      Mail ID\n'
            '   ',
  'subject': 'Вход с нового устройства в аккаунт'},
 {'data': '20.7.2020',
  'from': 'noreply@cloud.mail.ru',

            '  Записаться\n'
            '      Что такое Git и как его использовать?\n'
            '  12 ИЮНЯ, ПТ\n'
            '20:00 МСК\n'
            '  Записаться\n'
            '      Посмотреть все вебинары...\n'
            '  Сейчас!\n'
            '  Все вебинары\n'
            '            🔥 Распродажа\n'
            '  Скидка до 108 000 рублей на эти и другие факультеты.\n'
            '    Все факультеты со скидкой\n'
            '      За пределами GeekBrains\n'
            '  Наши статьи на интересных порталах, сайтах и в СМИ.\n'
            '    Игроделы бывают разные\n'
            'Разбираем, кто сейчас нужен в разработке игр и почему. Обзор '
            'зарплат прилагается. Хабр.\n'
            '        Декан говорит\n'
            'Сергей Ширкин (декан факультета Data Science) размышляет о '
            'будущем роботов.\n'
            '        Змей Горыныч\n'
            'Как живут и работают наставники в GeekBrains. А при чем тут змей '
            'с тремя г

            '  А вот так\n'
            '  Ну и по традиции: с Сергеем можно пообщаться в комментариях, а '
            'выучить Java-разработку — здесь.\n'
            '        Новое видео у нас на YouTube\n'
            '\n'
            'DevOps-инженер — кто это и как им стать?\n'
            '\n'
            'Смотреть сейчас\n'
            '        Вебинары на этой неделе\n'
            '    Метрики тестирования\n'
            '  09 ИЮНЯ, ВТ\n'
            '20:00 МСК\n'
            '  Записаться\n'
            '      Искусственный интеллект vs Программисты\n'
            '  10 ИЮНЯ, СР\n'
            '20:00 МСК\n'
            '  Записаться\n'
            '      Введение в моделирование данных\n'
            '  10 ИЮНЯ, СР\n'
            '20:00 МСК\n'
            '  Записаться\n'
            '      Технология ApplePay\n'
            '  11 ИЮНЯ, ЧТ\n'
            '19:00 МСК\n'
            '  Записаться\n'
            '      Что такое Git и как его использовать?\n'
            '  12 ИЮ

  'subject': 'Айтишник ---> Айтишник'},
 {'data': '9.6.2020',
  'from': 'digest@e.geekbrains.ru',
  'letter': '      Привет!\n'
            '  Сегодня у нас студент с нестандартной историей.\n'
            '\n'
            'Обычно переходят из НЕ-IT в IT. Это привычно. А Сергей Миненков '
            'перешел из 1С-программирования в Java-разработку. Из-за большой '
            'зарплаты? Неа.\n'
            '\n'
            'Сергею хотелось побольше заниматься техническими задачами, '
            'поменьше решать проблемы бизнеса.\n'
            '\n'
            'Несмотря на айтишную «базу», переход к джаве был тяжелым. Доход '
            'даже слегка снизился. Зато теперь Сергею все нравится в работе. И '
            'он даже добирается до нее в 2 раза быстрее :)\n'
            '\n'
            'И как так вышло?\n'
            '  А вот так\n'
            '  Ну и по традиции: с Сергеем можно пообщаться в комментариях, а '
            'выучить Java-разработку — здесь.\n'
            '

            '          P.S. Как вам письмо?\n'
            '    Понравилось!\n'
            '  Фиаско\n'
            '      Мы в социальных сетях\n'
            '    Свяжитесь с нами\n'
            '  sales@geekbrains.ru\n'
            '8-800-700-68-41\n'
            'Полезные ссылки\n'
            '  Бесплатные вебинары\n'
            'Обучающие курсы\n'
            'Тесты\n'
            'Наш блог\n'
            'Истории студентов\n'
            '      Вы получили это письмо, так как являетесь пользователем '
            'портала GeekBrains. Рассылка производится в соответствии с '
            'пользовательским соглашением. Если вы не хотите получать '
            'рассылку, вы можете отписаться или настроить параметры.\n'
            '  Если письмо отображается некорректно или вы хотите им '
            'поделиться, жмите сюда.\n'
            '  Общество с ограниченной ответственностью «ГикБреинс». ОГРН: '
            '1167746654569. Юридический адрес: 121205, г. Москва, территория '

            '19:00 МСК\n'
            '  Записаться\n'
            '      Что такое Git и как его использовать?\n'
            '  12 ИЮНЯ, ПТ\n'
            '20:00 МСК\n'
            '  Записаться\n'
            '      Посмотреть все вебинары...\n'
            '  Сейчас!\n'
            '  Все вебинары\n'
            '            🔥 Распродажа\n'
            '  Скидка до 108 000 рублей на эти и другие факультеты.\n'
            '    Все факультеты со скидкой\n'
            '      За пределами GeekBrains\n'
            '  Наши статьи на интересных порталах, сайтах и в СМИ.\n'
            '    Игроделы бывают разные\n'
            'Разбираем, кто сейчас нужен в разработке игр и почему. Обзор '
            'зарплат прилагается. Хабр.\n'
            '        Декан говорит\n'
            'Сергей Ширкин (декан факультета Data Science) размышляет о '
            'будущем роботов.\n'
            '        Змей Горыныч\n'
            'Как живут и работают наставники в GeekBrains. А при чем тут зме

            'И как так вышло?\n'
            '  А вот так\n'
            '  Ну и по традиции: с Сергеем можно пообщаться в комментариях, а '
            'выучить Java-разработку — здесь.\n'
            '        Новое видео у нас на YouTube\n'
            '\n'
            'DevOps-инженер — кто это и как им стать?\n'
            '\n'
            'Смотреть сейчас\n'
            '        Вебинары на этой неделе\n'
            '    Метрики тестирования\n'
            '  09 ИЮНЯ, ВТ\n'
            '20:00 МСК\n'
            '  Записаться\n'
            '      Искусственный интеллект vs Программисты\n'
            '  10 ИЮНЯ, СР\n'
            '20:00 МСК\n'
            '  Записаться\n'
            '      Введение в моделирование данных\n'
            '  10 ИЮНЯ, СР\n'
            '20:00 МСК\n'
            '  Записаться\n'
            '      Технология ApplePay\n'
            '  11 ИЮНЯ, ЧТ\n'
            '19:00 МСК\n'
            '  Записаться\n'
            '      Что такое Git и как его испо

            'Кроме того, РАТЭК предложила провести тестирование закона: в '
            'течение года устанавливать на ряд гаджетов только одно '
            'приложение, к примеру, «Госуслуги», и посмотреть, какие всплывут '
            'проблемы.\n'
            'Ваши разговоры записываются\n'
            'Стартап Voca.Tech и «Яндекс» создали бейджи для записи и анализа '
            'речи, Voca.Badge. Он нацелен на бизнес. Предполагается, что '
            'компании будут с помощью бейджей анализировать общение сотрудника '
            'с клиентом, насколько он был вежлив, какую предоставлял '
            'информацию о товаре и так далее.\n'
            'Бейдж способен вести запись на протяжении 12 часов. Данные он '
            'передаёт по Wi-Fi на серверы Яндекс.Облака. Там речь анализирует '
            'Yandex SpeechKit.\n'
            'Устройство обойдётся компаниям в 9,5 тысяч рублей, а его '
            'обслуживание — в 999 рублей в месяц.\n'
            'На русском\n'
     

            '____________\n'
            '\n'
            'Катя\n'
            'Поделиться\n'
            'Твитнуть\n'
            'Переслать\n'
            'Вы получаете эти письма, потому что подписались на рассылку '
            'Tproger.\n'
            'Отписаться от рассылки.',
  'subject': 'Обзор IT-новостей 20 декабря'},
 {'data': '20.12.2019',
  'from': 'news@tproger.ru',
  'letter': 'Интернет\n'
            'Минкомсвязи запланировало на 23 декабря учения по автономному '
            'Рунету: операторы, провайдеры и несколько федеральных служб '
            'проверят, насколько Рунет готов к катаклизмам. Власти обещают, '
            'что обычные пользователи этого даже не заметят.\n'
            'Как водится, за кулисами новости происходит много чего '
            'интересного, об этом рассказал Филипп Кулин. Он пристально следит '
            'темой автономного Рунета и в своём Telegram-канале «Эшер II» '
            'выкладывает и анализирует все материалы, какие только найд

            'Беспилотного полку прибыло: «КамАЗ» начал тестировать на '
            'производстве беспилотные грузовики. Они перевозят кабины с '
            'прессово-рамного завода на автомобильный. Машины оборудованы как '
            'надо: лидарами, радарами, видеокамерами, ультразвуковыми '
            'датчиками, а ещё модулями Wi-Fi и 4G-связи.\n'
            '____________\n'
            '\n'
            'Катя Никитина\n'
            'Поделиться\n'
            'Твитнуть\n'
            'Переслать\n'
            'Вы получаете эти письма, потому что подписались на рассылку '
            'Tproger.\n'
            'Отписаться от рассылки.',
  'subject': 'Вечерний обзор IT-новостей 3 декабря'}]


In [12]:
client = MongoClient('localhost:27017')
db = client.mail
collection = db.mailru
try:
    collection.drop()
    collection.insert_many(list_letters)
except Exception as e:
    print(e)

In [13]:
df = pd.DataFrame(list_letters)
df

,data,from,subject,letter,_id
0,30.07.2020,security@id.mail.ru,Вход с нового устройства в аккаунт,Вход с нового устройства в аккаунт\n s...,5f22facf9693a1565fbe81bf
1,20.7.2020,noreply@cloud.mail.ru,Добро пожаловать в Облако,Здравствуйте!\n Добро пожаловать в Обла...,5f22facf9693a1565fbe81c0
2,14.7.2020,digest@e.geekbrains.ru,"<Python> Вакансии, вебинары и ЗП до 200 000 ру...",Привет!\n Собрали вам несколько интерес...,5f22facf9693a1565fbe81c1
3,11.7.2020,service@e.mail.ru,1 год вместе! Расскажите – вам всё нравится?,Здравствуйте!\n Вашему ящику исполнился...,5f22facf9693a1565fbe81c2
4,25.6.2020,study.ai_172@mail.ru,Ночная тема,Сон для слабаков\n --\nstudy ai_172,5f22facf9693a1565fbe81c3
...,...,...,...,...,...
113,5.12.2019,news@tproger.ru,Вечерний обзор IT-новостей 5 декабря,Распознавание с подвохом\nИздание «МБХ Медиа» ...,5f22facf9693a1565fbe8230
114,5.12.2019,digest@e.geekbrains.ru,𝐆𝐄𝐄𝐊 𝐌𝐎𝐑𝐎𝐙,Если это электронное сообщение отображается не...,5f22facf9693a1565fbe8231
115,4.12.2019,security@e.mail.ru,Изменён пароль в аккаунте,Изменён пароль в аккаунте\n study.ai_1...,5f22facf9693a1565fbe8232
116,4.12.2019,news@tproger.ru,Вечерний обзор IT-новостей 4 декабря,Всё кончается\nСергей Брин и Ларри Пейдж остав...,5f22facf9693a1565fbe8233


In [14]:
driver.close()